In [1]:
import os
import json
import random
import subprocess
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
projects = ["JPetStore"]  # ["JPetStore", "DayTrader", "AcmeAir", "Plants"]
for proj in projects:
	with open(f"results/Mo2oM/Mo2oM_{proj}.json") as f:
		data = json.load(f)
	df = pd.DataFrame(data)
	ddf = df.drop(columns=["microservices"])
	normalized_df = pd.DataFrame(MinMaxScaler().fit_transform(ddf.values), columns=ddf.columns)
	df['score'] = normalized_df["SM"] - normalized_df["ICP"] - normalized_df["IFN"] - normalized_df["NED"]
	idx = df["score"].argmax()
	print(df.iloc[idx])
	classes_microservices = df["microservices"].iloc[idx]
	libs = os.path.join("..", "Mo2oM", "JavaParser", "lib", "javaparser-core-3.25.5-SNAPSHOT.jar") + os.pathsep + \
		   os.path.join("..", "Mo2oM", "JavaParser", "lib", "json-20230618.jar")
	json_path = os.path.join("..", "Mo2oM", "JavaParser", "classes.json")
	source_path = os.path.join("..", "test_projects", proj, "OneFileSource.java")
	parser_path = os.path.join("..", "Mo2oM", "JavaParser", "Parser.java")
	subprocess.run(['java', '-cp', libs, parser_path, source_path, json_path])
	with open(json_path, "rt") as classes_file:
		classes_info = json.load(classes_file)
	for clss in classes_info:
		for call in classes_info[clss]["method_calls"]:
			for other_clss in classes_info:
				if call["method_name"] in classes_info[other_clss]["methods"]:
					call["class_name"] = other_clss
					if clss == other_clss:
						break
	class_names = list(classes_info.keys())

	calls = []
	for caller_class_i, caller_class_n in enumerate(classes_info):
		caller_microservices = classes_microservices[caller_class_i]
		if caller_microservices != [-1]:
			for call in classes_info[caller_class_n]["method_calls"]:
				if call["class_name"] in class_names:
					callee_microservices = classes_microservices[class_names.index(call["class_name"])]
					if callee_microservices != [-1]:
						for caller_microservice in caller_microservices:
							if caller_microservice in callee_microservices:
								calls.append({
									"caller_microservice": f"microservice_{caller_microservice}",
									"caller_class": caller_class_n,
									"callee_microservice": f"microservice_{caller_microservice}",
									"callee_class": call["class_name"],
								})
							else:
								calls.append({
									"caller_microservice": f"microservice_{caller_microservice}",
									"caller_class": caller_class_n,
									"callee_microservice": f"microservice_{random.choice(callee_microservices)}",
									"callee_class": call["class_name"],
								})
# 	mfs = []
# 	for clss in class_names:
# 		clss_ms_list = classes_microservices[class_names.index(clss)]
# 		if len(clss_ms_list) > 1:
# 			mfs.append(clss)
# print(mfs)
# print(len(mfs))
# print(len(class_names))
df = pd.DataFrame(calls)
df

n_clusters                                                       9
alpha                                                          0.4
threshold                                                     0.25
microservices    [[1, 3, 4, 6], [6], [-1], [-1], [-1], [-1], [-...
SM                                                        0.530189
ICP                                                        0.13694
IFN                                                       1.555556
NED                                                       0.013158
score                                                     0.328953
Name: 1353, dtype: object


,caller_microservice,caller_class,callee_microservice,callee_class
0,microservice_6,OracleSequenceDao,microservice_6,Category
1,microservice_6,OracleSequenceDao,microservice_6,OracleSequenceDao
2,microservice_5,Item,microservice_5,Item
3,microservice_7,Item,microservice_7,Item
4,microservice_5,Item,microservice_5,Item
...,...,...,...,...
730,microservice_4,ViewOrderAction,microservice_4,Order
731,microservice_4,SqlMapAccountDao,microservice_4,Order
732,microservice_4,SqlMapAccountDao,microservice_4,AccountActionForm
733,microservice_4,SqlMapAccountDao,microservice_4,AccountActionForm


In [3]:
df = df[(df["caller_class"] != df["callee_class"]) | (df["caller_microservice"] != df["callee_microservice"])]
df

,caller_microservice,caller_class,callee_microservice,callee_class
0,microservice_6,OracleSequenceDao,microservice_6,Category
6,microservice_1,AccountActionForm,microservice_1,Order
7,microservice_2,AccountActionForm,microservice_2,Order
8,microservice_3,AccountActionForm,microservice_3,Order
9,microservice_4,AccountActionForm,microservice_4,Order
...,...,...,...,...
730,microservice_4,ViewOrderAction,microservice_4,Order
731,microservice_4,SqlMapAccountDao,microservice_4,Order
732,microservice_4,SqlMapAccountDao,microservice_4,AccountActionForm
733,microservice_4,SqlMapAccountDao,microservice_4,AccountActionForm


In [9]:
clss_score = []
for clss in class_names:
	this_df = df[(df["caller_class"] == clss) | (df["callee_class"] == clss)]
	if this_df.shape[0]:
		intra = this_df[this_df["caller_microservice"] == this_df["callee_microservice"]]
		clss_score.append({
			"class": clss,
			"score": (intra.shape[0] / this_df.shape[0])
		})
sorted_clss_score = sorted(clss_score, key=lambda x: x["score"], reverse=True)
print(len(sorted_clss_score))
keep_classes = [x["class"] for x in sorted_clss_score[:-3]]
keep_classes.remove("SqlMapOrderDao")
print(len(keep_classes))
tmp_df = df[(df["caller_class"].isin(keep_classes)) & (df["callee_class"].isin(keep_classes))]
tmp_df.reset_index(drop=True, inplace=True)
mfs = []
for clss in keep_classes:
	clss_ms_list = set(tmp_df[tmp_df["caller_class"] == clss]["caller_microservice"].unique()).union(set(tmp_df[tmp_df["callee_class"] == clss]["callee_microservice"].unique()))
	if len(clss_ms_list) > 1:
		mfs.append(clss)
to_remove = []
n_f = 0
for clss in keep_classes:
	if clss in mfs:
		if n_f < 5:
			n_f += 1
		else:
			to_remove.append(clss)
keep_classes = [x for x in keep_classes if x not in to_remove]
print(len(keep_classes))
new_df = df[(df["caller_class"].isin(keep_classes)) & (df["callee_class"].isin(keep_classes))]
new_df = new_df.drop(new_df[(new_df["callee_microservice"] == "microservice_1") | (new_df["caller_microservice"] == "microservice_1")].index)
new_df.reset_index(drop=True, inplace=True)
new_df

43
39
25


,caller_microservice,caller_class,callee_microservice,callee_class
0,microservice_6,OracleSequenceDao,microservice_6,Category
1,microservice_0,AddItemToCartAction,microservice_0,BaseAction
2,microservice_0,AddItemToCartAction,microservice_0,BaseAction
3,microservice_4,NewOrderFormAction,microservice_4,SqlMapAccountDao
4,microservice_4,NewOrderFormAction,microservice_4,BaseAction
5,microservice_4,NewOrderFormAction,microservice_4,SqlMapAccountDao
6,microservice_4,ListOrdersController,microservice_4,SqlMapAccountDao
7,microservice_0,SearchProductsAction,microservice_0,BaseAction
8,microservice_4,SignonAction,microservice_4,BaseAction
9,microservice_4,SignonAction,microservice_4,SqlMapAccountDao


In [10]:
# row_counts = df.groupby(df.columns.tolist()).size()
# rows_to_keep = row_counts[row_counts >= 3].index
# new_df = df[df[df.columns].apply(tuple, axis=1).isin(rows_to_keep)]
# new_df.reset_index(drop=True, inplace=True)
# new_df

In [11]:
jdata = []
microservices = new_df["caller_microservice"].unique()
for ms in microservices:
	classes = new_df[new_df["caller_microservice"] == ms]["caller_class"].unique()
	for clss in classes:
		calls = new_df[(new_df["caller_microservice"] == ms) & (new_df["caller_class"] == clss)]
		imports = []
		for v in calls.values:
			if v[2] != ms or v[3] != clss:
				imports.append(projects[0]+"."+v[2]+"."+v[3])
		jdata.append({"name": projects[0]+"."+ms+"."+clss, "imports": imports})
no_imports = set()
for jd in jdata:
	for imp in jd["imports"]:
		for jd in jdata:
			if jd["name"] == imp:
				break
		else:
			no_imports.add(imp)
for ni in no_imports:
	jdata.append({"name": ni, "imports": []})
with open(f"Mo2oM_{projects[0]}_calls.json", "w") as f:
	json.dump(jdata, f)

In [12]:
selected_classes = set(new_df["caller_class"].unique()).union(set(new_df["callee_class"].unique()))
selected_classes

{'Account',
 'AddItemToCartAction',
 'BaseAction',
 'Category',
 'EditAccountAction',
 'EditAccountFormAction',
 'ListOrdersAction',
 'ListOrdersController',
 'NewAccountFormAction',
 'NewOrderFormAction',
 'OracleSequenceDao',
 'SearchProductsAction',
 'SendOrderConfirmationEmailAdvice',
 'SignonAction',
 'SignonController',
 'SqlMapAccountDao',
 'ViewOrderAction'}

In [13]:
r = []
for clss in selected_classes:
	l = set(new_df[new_df["caller_class"]==clss]["caller_microservice"].unique()).union(set(new_df[new_df["callee_class"]==clss]["callee_microservice"].unique()))
	if len(l) > 1:
		r.append(clss)
print(r)

['BaseAction', 'Account']
